In [ ]:
!!pip install datasets

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting datasets',
 '  Downloading datasets-2.4.0-py3-none-any.whl (365 kB)',
 '\x1b[?25l',
 '\x1b[K     |█                               | 10 kB 15.9 MB/s eta 0:00:01',
 '\x1b[K     |█▉                              | 20 kB 8.1 MB/s eta 0:00:01',
 '\x1b[K     |██▊                             | 30 kB 8.5 MB/s eta 0:00:01',
 '\x1b[K     |███▋                            | 40 kB 5.1 MB/s eta 0:00:01',
 '\x1b[K     |████▌                           | 51 kB 5.0 MB/s eta 0:00:01',
 '\x1b[K     |█████▍                          | 61 kB 5.9 MB/s eta 0:00:01',
 '\x1b[K     |██████▎                         | 71 kB 5.9 MB/s eta 0:00:01',
 '\x1b[K     |███████▏                        | 81 kB 6.6 MB/s eta 0:00:01',
 '\x1b[K     |████████                        | 92 kB 5.1 MB/s eta 0:00:01',
 '\x1b[K     |█████████                       | 102 kB 5.5 MB/s eta 0:00:01',
 '\x1b[K     |█████████▉    

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [117]:
import random
import numpy as np
import math

import nltk
import gensim.downloader as api

import torch
import torch.nn as nn
import datasets

In [ ]:
SEED = 0
random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)

In [ ]:
dataset = datasets.load_dataset("ag_news")

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
print("\n".join(api.info()["models"].keys()))

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [ ]:
word2vec = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


In [ ]:
MAX_LENGTH=128

tokenizer = nltk.WordPunctTokenizer()

dataset = dataset.map(
    lambda item: {
        "tokenized": tokenizer.tokenize(item["text"])[:MAX_LENGTH]
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [ ]:
dataset["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'tokenized': ['Wall',
  'St',
  '.',
  'Bears',
  'Claw',
  'Back',
  'Into',
  'the',
  'Black',
  '(',
  'Reuters',
  ')',
  'Reuters',
  '-',
  'Short',
  '-',
  'sellers',
  ',',
  'Wall',
  'Street',
  "'",
  's',
  'dwindling',
  '\\',
  'band',
  'of',
  'ultra',
  '-',
  'cynics',
  ',',
  'are',
  'seeing',
  'green',
  'again',
  '.']}

In [ ]:
word2idx = {word: idx for idx, word in enumerate(word2vec.index2word)}

In [ ]:
def encode(word):
    if word in word2idx.keys():
        return word2idx[word]
    return word2idx["unk"]

In [ ]:
dataset = dataset.map(
    lambda item: {
        "features": [encode(word) for word in item["tokenized"]]
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [ ]:
dataset["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'tokenized': ['Wall',
  'St',
  '.',
  'Bears',
  'Claw',
  'Back',
  'Into',
  'the',
  'Black',
  '(',
  'Reuters',
  ')',
  'Reuters',
  '-',
  'Short',
  '-',
  'sellers',
  ',',
  'Wall',
  'Street',
  "'",
  's',
  'dwindling',
  '\\',
  'band',
  'of',
  'ultra',
  '-',
  'cynics',
  ',',
  'are',
  'seeing',
  'green',
  'again',
  '.'],
 'features': [62980,
  62980,
  1,
  62980,
  62980,
  62980,
  62980,
  13,
  62980,
  17,
  62980,
  20,
  62980,
  28,
  62980,
  28,
  49286,
  4,
  62980,
  62980,
  48,
  137,
  214902,
  370,
  1645,
  39,
  8606,
  28,
  380053,
  4,
  70,
  1321,
  1745,
  389,
  1]}

In [ ]:
dataset = dataset.remove_columns(["text", "tokenized"])

In [ ]:
dataset.set_format(type='torch')

In [ ]:
dataset["train"][0]

{'label': tensor(2),
 'features': tensor([ 62980,  62980,      1,  62980,  62980,  62980,  62980,     13,  62980,
             17,  62980,     20,  62980,     28,  62980,     28,  49286,      4,
          62980,  62980,     48,    137, 214902,    370,   1645,     39,   8606,
             28, 380053,      4,     70,   1321,   1745,    389,      1])}

In [ ]:
def collate_fn(batch):
    max_len = max(len(row["features"]) for row in batch)
    input_embeds = torch.empty((len(batch), max_len), dtype=torch.long)
    labels = torch.empty(len(batch), dtype=torch.long)
    for idx, row in enumerate(batch):
        to_pad = max_len - len(row["features"])
        input_embeds[idx] = torch.cat((row["features"], torch.zeros(to_pad)))
        labels[idx] = row["label"]
    return {"features": input_embeds, "labels": labels}

In [ ]:
from torch.utils.data import DataLoader

loaders = {
    k: DataLoader(
        ds, shuffle=(k=="train"), batch_size=32, collate_fn=collate_fn
    ) for k, ds in dataset.items()
}

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, embedding_dim: int, hidden_dim: int, num_classes: int = 4):
        super(CNNModel, self).__init__()
        self.embeddings = nn.Embedding(len(word2idx), embedding_dim=embedding_dim)
        self.cnn = nn.Sequential(
            nn.Conv1d(embedding_dim, hidden_dim, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1),
            nn.Flatten()
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embeddings(x)
        x = x.permute(0, 2, 1)
        x = self.cnn(x)
        x = self.fc(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
model = CNNModel(word2vec.vector_size, 50).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 1

In [ ]:
from tqdm import tqdm, trange

def training(model, criterion, optimizer, epochs, loaders, max_grad_norm=2):
    for epoch in trange(epochs, leave=False):
        print(f"Training: {epoch}/{epochs}")
        model.train()
        iters = 0
        pbar = tqdm(loaders["train"], leave=True)
        for batch in pbar:
            optimizer.zero_grad()

            features = batch["features"].to(device)
            labels = batch["labels"].to(device)
            
            preds = model(features)
            loss = criterion(preds, labels)

            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            iters += 1
        valid_loss = 0
        valid_acc = 0
        iters = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for batch in loaders["test"]:

                features = batch["features"].to(device)
                labels = batch["labels"].to(device)
                
                preds = model(features)
                loss = criterion(preds, labels)

                valid_loss += loss.item()
                correct += (labels == preds.argmax(-1)).float().sum()
                total += len(labels)
                iters += 1

        print(f"Valid Loss: {valid_loss / iters}, accuracy: {correct/total}")

In [ ]:
training(model, criterion, optimizer, epochs, loaders)

  0%|          | 0/1 [00:00<?, ?it/s]

Training: 0/1



100%|██████████| 3750/3750 [01:54<00:00, 32.67it/s]
                                              

Valid Loss: 0.49331000841715755, accuracy: 0.8236841559410095


In [ ]:
class RNNCell(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int):
        super(RNNCell, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        self.w_h = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_h = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_x = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_x = nn.Parameter(torch.rand(1, hidden_dim))

    def forward(self, x, hidden=None):
        '''
        x – torch.FloatTensor with the shape (batch, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        seq_len = x.size(1)
        for cur_idx in range(seq_len):
            hidden = torch.tanh(
                x[:, cur_idx] @ self.w_x + self.b_x + hidden @ self.w_h + self.b_h
            )
        return hidden

In [ ]:
class RNN(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int, num_classes: int = 4):
        super(RNN, self).__init__()
        self.embeddings = nn.Embedding(len(word2idx), embed_dim)
        self.rnn_cell = RNNCell(embed_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.rnn_cell(x)
        x = self.fc(x)
        return x

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = RNN(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

epochs = 1
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

Training: 0/1



100%|██████████| 3750/3750 [03:25<00:00, 18.23it/s]
                                              

Valid Loss: 1.4122406870377164, accuracy: 0.2499999850988388


In [ ]:
class GRUCell(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int):
        super(GRUCell, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        self.w_rh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_rh = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_rx = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_rx = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_zh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_zh = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_zx = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_zx = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_nh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_nh = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_nx = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_nx = nn.Parameter(torch.rand(1, hidden_dim))

    def forward(self, x, hidden=None):
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        seq_len = x.size(1)
        for cur_idx in range(seq_len):
            r = torch.sigmoid(
                x[:, cur_idx] @ self.w_rx + self.b_rx + hidden @ self.w_rh + self.b_rh
            )
            z = torch.sigmoid(
                x[:, cur_idx] @ self.w_zx + self.b_zx + hidden @ self.w_zh + self.b_zh
            )
            n = torch.tanh(
                x[:, cur_idx] @ self.w_nx + self.b_nx + r * (hidden @ self.w_nh + self.b_nh)
            )
            hidden = (1 - z) * n + z * hidden
        return hidden

In [ ]:
class GRU(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embed = nn.Embedding(len(word2idx), embed_size)
        self.gru = GRUCell(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embed(x)
        hidden = self.gru(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRU(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

epochs = 1
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

Training: 0/1



100%|██████████| 3750/3750 [07:56<00:00,  7.87it/s]
                                              

Valid Loss: 0.6836419919458758, accuracy: 0.7438157796859741


In [ ]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:172.)
  after removing the cwd from sys.path.


In [ ]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embed
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

In [ ]:
def training_freeze(model, criterion, optimizer, epochs, loaders, max_grad_norm=2, num_freeze_iter=1000):
    freeze_embeddings(model)
    for e in trange(epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False)
        for batch in pbar:
            if num_iter > num_freeze_iter and e < 1:
                freeze_embeddings(model, True)
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_objs += len(labels)
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs}")

In [ ]:
training_freeze(model, criterion, optimizer, epochs, loaders, max_grad_norm)

100%|██████████| 3750/3750 [07:09<00:00,  7.54it/s]
                                              

Valid Loss: 0.44941553473472595, accuracy: 0.8511841893196106


In [ ]:
class LSTMCell_v0(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int):
        super(LSTMCell_v0, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        self.w_fh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_fh = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_fx = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_fx = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_ix = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_ix = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_ih = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_ih = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_gx = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_gx = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_gh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_gh = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_ox = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_ox = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_oh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_oh = nn.Parameter(torch.rand(1, hidden_dim))
        self.init_weights()

    def init_weights(self):
        import math
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, x, cell_state=None, hidden_state=None):
        if cell_state is None:
            cell_state = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        if hidden_state is None:
            hidden_state = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        
        f = torch.sigmoid(x @ self.w_fx + self.b_fx + hidden_state @ self.w_fh + self.b_fh)
        i = torch.sigmoid(x @ self.w_ix + self.b_ix + hidden_state @ self.w_ih + self.b_ih)
        g = torch.tanh(x @ self.w_gx + self.b_gx + hidden_state @ self.w_gh + self.b_gh)
        output_gate = torch.sigmoid(
            x @ self.w_ox + self.b_ox + hidden_state @ self.w_oh + self.b_oh
            ) * torch.tanh(cell_state)
        cell_state = f * cell_state + i * g
        hidden_state = output_gate * torch.tanh(cell_state)
        return hidden_state, (hidden_state, cell_state)

In [125]:
class LSTMCell_v1(nn.Module):
    """Vectorized version"""
    def __init__(self, embed_dim: int, hidden_dim: int):
        super(LSTMCell_v1, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
     
        self.W = nn.Parameter(torch.rand(embed_dim, hidden_dim * 4))
        self.U = nn.Parameter(torch.rand(hidden_dim, hidden_dim * 4))
        self.bias = nn.Parameter(torch.rand(1, hidden_dim * 4))

        self.init_weights()

    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, x, cell_state=None, hidden_state=None):
        if cell_state is None:
            cell_state = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        if hidden_state is None:
            hidden_state = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        seq_len = x.size(1)
        outputs = []
        for cur_idx in range(seq_len):
            gates = x[:, cur_idx] @ self.W + hidden_state @ self.U + self.bias
            i, f, g, o = (
                torch.sigmoid(gates[:, :self.hidden_dim]), # input
                torch.sigmoid(gates[:, self.hidden_dim:2 * self.hidden_dim]), # forget
                torch.tanh(gates[:, 2 * self.hidden_dim:3 * self.hidden_dim]),
                torch.sigmoid(gates[:, 3 * self.hidden_dim:]), # output
            )
            output_gate = o * torch.tanh(cell_state)
            cell_state = f * cell_state + i * g
            hidden_state = output_gate * torch.tanh(cell_state)

            outputs.append(hidden_state.unsqueeze(1))

        outputs = torch.cat(outputs, dim=1).contiguous()

        return outputs, (hidden_state, cell_state)


class LSTM(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int, classes: int = 4):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embed = nn.Embedding(len(word2idx), embedding_dim=embed_dim)
        self.lstm = LSTMCell_v1(embed_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, classes)

    def forward(self, x):
        x = self.embed(x)
        output, (hidden_state, cell_state) = self.lstm(x)
        output = self.fc(hidden_state)
        return output

In [129]:
class LSTMCell_v2(nn.Module):
    """Vectorized version"""
    def __init__(self, embed_dim: int, hidden_dim: int):
        super(LSTMCell_v1, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        self.w_fh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_fh = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_fx = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_fx = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_ix = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_ix = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_ih = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_ih = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_gx = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_gx = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_gh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_gh = nn.Parameter(torch.rand(1, hidden_dim))

        self.w_ox = nn.Parameter(torch.rand(embed_dim, hidden_dim))
        self.b_ox = nn.Parameter(torch.rand(1, hidden_dim))
        self.w_oh = nn.Parameter(torch.rand(hidden_dim, hidden_dim))
        self.b_oh = nn.Parameter(torch.rand(1, hidden_dim))
        self.init_weights()

    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, x, cell_state=None, hidden_state=None):
        if cell_state is None:
            cell_state = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        if hidden_state is None:
            hidden_state = torch.zeros((x.size(0), self.hidden_dim)).to(device)
        seq_len = x.size(1)
        outputs = []
        for cur_idx in range(seq_len):
        
            f = torch.sigmoid(x[:, cur_idx] @ self.w_fx + self.b_fx + hidden_state @ self.w_fh + self.b_fh)
            i = torch.sigmoid(x[:, cur_idx] @ self.w_ix + self.b_ix + hidden_state @ self.w_ih + self.b_ih)
            g = torch.tanh(x[:, cur_idx] @ self.w_gx + self.b_gx + hidden_state @ self.w_gh + self.b_gh)
            output_gate = torch.sigmoid(
                x[:, cur_idx] @ self.w_ox + self.b_ox + hidden_state @ self.w_oh + self.b_oh
                ) * torch.tanh(cell_state)
            cell_state = f * cell_state + i * g
            hidden_state = output_gate * torch.tanh(cell_state)

            outputs.append(hidden_state.unsqueeze(1))

        outputs = torch.cat(outputs, dim=1).contiguous()

        return outputs, (hidden_state, cell_state)


class LSTM(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int, classes: int = 4):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embed = nn.Embedding(len(word2idx), embedding_dim=embed_dim)
        self.lstm = LSTMCell_v1(embed_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, classes)

    def forward(self, x):
        x = self.embed(x)
        output, (hidden_state, cell_state) = self.lstm(x)
        output = self.fc(hidden_state)
        return output

In [110]:
class LSTM_Pytorch_V1(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int, classes: int = 4):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embed = nn.Embedding(len(word2idx), embedding_dim=embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, 1, batch_first=True)
        self.fc = nn.Linear(hidden_dim, classes)

    def forward(self, x):
        x = self.embed(x)
        
        output, (hidden_state, cell_state) = self.lstm(x)
        output = self.fc(hidden_state.squeeze(0))
        return output

In [130]:
model = LSTM(word2vec.vector_size, 50).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 1

In [131]:
def training(model, criterion, optimizer, epochs, loaders, max_grad_norm=2):
    for epoch in trange(epochs, leave=False):
        print(f"Training: {epoch}/{epochs}")
        model.train()
        iters = 0
        pbar = tqdm(loaders["train"], leave=True)
        for batch in pbar:
            optimizer.zero_grad()

            features = batch["features"].to(device)
            labels = batch["labels"].to(device)

            preds = model(features)
            loss = criterion(preds, labels)

            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            iters += 1
        valid_loss = 0
        valid_acc = 0
        iters = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for batch in loaders["test"]:

                features = batch["features"].to(device)
                labels = batch["labels"].to(device)
                
                preds = model(features)
                loss = criterion(preds, labels)

                valid_loss += loss.item()
                correct += (labels == preds.argmax(-1)).float().sum()
                total += len(labels)
                iters += 1

        print(f"Valid Loss: {valid_loss / iters}, accuracy: {correct/total}")

In [132]:
training(model, criterion, optimizer, epochs, loaders)

  0%|          | 0/1 [00:00<?, ?it/s]

Training: 0/1



100%|██████████| 3750/3750 [06:08<00:00, 10.18it/s]
                                              

Valid Loss: 0.7541358343192509, accuracy: 0.7153946757316589
